## Блокнот для прикидки влияния отдельных параметров


In [1]:
# from utils import X,y,cat_cols,fillna,normalize,skew,encode
from task import *
import gc
gc.collect()

1131

### Для тестирования используется тренировочный набор
Набор разделён на обучающий и тестовый  в соотношении 3:1 без перемешивания 

In [2]:
def data():
    split = 300_000
    maxlines = 500_000
    d2 = pd.read_csv('datasets/train.csv',skiprows= range(1,split) )
    d1 = pd.read_csv('datasets/train.csv',skiprows= range(split,maxlines) )
    return(d1,d2)  


### Класс для прогона тестов
На экран выводится метрика MAPE на тетсовом наборе 
Кроме того, базовый класс записывает метрику в лог, откуда её можно извлечь

In [3]:
class Validator(Task):

    def __init__( self, data, params = {}):
        super().__init__(data, params)
        self.ds = ScoreSet( data[0],data[1] ,params=self.params)

    def process(self,ds = None ):
        if ds==None:
            ds = self.ds
        ds.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        ds.prepare().fit().predict()
        self.log +=(f"{ds.log}\t MAPE:{ds.scores()}\n")          

    

In [4]:
# params = { 
#             'rounding': { 'odometer':3000,'condition':1},
# #            'skew': {'threshold':4000,'mult':4 },
#             'model': {  'criterion':'poisson','n_estimators':500, 'max_depth':200, 'min_samples_leaf':2 }  
#         }
# v = Validator( data() ,params=params)
# v.process()
# print(v.log)
# del v
# gc.collect()

### Подбор параметров в цикле

In [5]:
params = { 
            'rounding': { 'odometer':3000,'condition':1},
            'model': {  'criterion':'poisson','n_estimators':200, 'max_depth':50, 'min_samples_leaf':2 }  
        }
for odo in [1_000,3_000]:
    for th in [2500,5000]:
        for mult in [1,2,4,8]:
            params.update({'rounding': { 'odometer':odo ,'condition':1}, 'skew': {'threshold':th,'mult':mult }})
            v = Validator( data() ,params=params)
            v.process()
            print(v.log)       
# -- interrupted on step 11 as no further improvements expected             
            

	 MAPE:0.17171102086057927

	 MAPE:0.17513391544669243

	 MAPE:0.179693614575781

	 MAPE:0.18173572352205006

	 MAPE:0.17178266623484453

	 MAPE:0.17587527502513864

	 MAPE:0.18085793468476952

	 MAPE:0.1827270585977118

	 MAPE:0.17165868596827347

	 MAPE:0.17530877372252246

	 MAPE:0.17967759740570602



KeyboardInterrupt: 

In [6]:
# Unfortunately data size proved to be too big for my PC, so I have to gree memory as often as possibly :(
import gc
gc.collect()

6619